In [1]:
import threading
import time

In [2]:
# Thread, Timer, Event and Lock. Those are the bits you'll need.

# For the non-blocking moves the basic idea is that you write a function that contains the 
# while loop that checks when it's time to stop the move, you create a Thread and pass it 
# that function, start the move, start the Thread, then return. The Thread will run in the 
# background and stop the move then at target. (edited)

# You can use Events to do simple communication between threads, e.g. you can pass one to 
# the dome movement thread and use it to stop the move early if the abort function is called.

In [3]:
import threading
import time

class FakeDome(object):
    def __init__(self):
        self._encoder_count = 0
        self.abort_event = threading.Event()

    @property
    def encoder_count(self):
        return self._encoder_count

    def move(self, ticks=10):
        # dummy move command that just moves us 10 ticks from current encoder position
        target_count = self.encoder_count + ticks
        # create the monitor thread that tracks completion of movement command
        self.monitor = threading.Thread(target=self.move_condition, args=[target_count])
        # create a thread to simulate ticks
        self.simticks = threading.Thread(target=self.simulate_ticks, args=[ticks])
        # start both threads
        self.monitor.start()
        self.simticks.start()
        return
    
    def move_condition(self, target_count):
        # record the start time, to check for timeout condition
        start = time.time()
        while self.encoder_count < target_count:
            # check to see if abort_event is set and end move() if it is
            if self.abort_event.is_set():
                return
            # short break to space ticks/status-checks out
            time.sleep(1)
            print(f'encoder count recorded, {target_count - self.encoder_count} ticks remaining.')
            # if monitor thread has been going for more than 15s end it
            if time.time() - start > 15:
                return
        return
    
    def simulate_ticks(self, ticks):
        for tick in range(ticks):
            # check to see if abort_event is set and end move() if it is
            if self.abort_event.is_set():
                return
            time.sleep(1)
            print(f'ENCODER COUNT INCREMENT, current count: {self.encoder_count}')
            self._encoder_count+=1
        return
    
    def abort_move(self):
        # set the abort_event flag
        self.abort_event.set()
        # check to make sure both the monitor and simticks thread have terminated
        if not self.monitor.is_alive() and not self.simticks.is_alive():
            # now we can reset the abort_event flag
            self.abort_event.clear()
        return


In [4]:
fdome = FakeDome()

In [5]:
fdome.move()

encoder count recorded, 10 ticks remaining.
ENCODER COUNT INCREMENT, current count: 0
ENCODER COUNT INCREMENT, current count: 1
encoder count recorded, 8 ticks remaining.
ENCODER COUNT INCREMENT, current count: 2
encoder count recorded, 7 ticks remaining.
ENCODER COUNT INCREMENT, current count: 3
encoder count recorded, 6 ticks remaining.
ENCODER COUNT INCREMENT, current count: 4
encoder count recorded, 5 ticks remaining.


In [6]:
fdome.abort_move()

ENCODER COUNT INCREMENT, current count: 5
encoder count recorded, 4 ticks remaining.


In [6]:
fdome._encoder_count = 0

In [7]:
fdome.encoder_count

0

In [7]:
fdome.move()